##### Copyright 2021 The TF-Agents Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Checkpointer 및 PolicySaver

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/10_checkpointer_policysaver_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/10_checkpointer_policysaver_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/10_checkpointer_policysaver_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">깃허브(GitHub) 소스 보기</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/10_checkpointer_policysaver_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 소개

`tf_agents.utils.common.Checkpointer`는 훈련 상태, 정책 상태 및 replay_buffer 상태를 로컬 스토리지로/에서 저장/로드하는 유틸리티입니다.

`tf_agents.policies.policy_saver.PolicySaver`는 정책만 저장/로드하는 도구이며 `Checkpointer`보다 가볍습니다. 정책이 생성된 코드를 몰라도 `PolicySaver`를 사용하여 모델을 배포할 수도 있습니다.

이 튜토리얼에서는 DQN을 사용하여 모델을 훈련한 다음, `Checkpointer` 및 `PolicySaver`를 사용하여 대화식 방식으로 상태 및 모델을 저장하고 로드하는 방법을 보여줍니다. `PolicySaver`에 TF2.0의 새로운 saved_model 도구 및 형식을 사용할 것입니다.


## 설정

다음과 같은 종속성을 설치하지 않은 경우, 다음을 실행합니다.

In [ ]:
#@test {"skip": true}
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg python-opengl
!pip install pyglet
!pip install 'imageio==2.4.0'
!pip install 'xvfbwrapper==0.2.9'
!pip install tf-agents[reverb]

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import imageio
import io
import matplotlib
import matplotlib.pyplot as plt
import os
import shutil
import tempfile
import tensorflow as tf
import zipfile
import IPython

try:
  from google.colab import files
except ImportError:
  files = None
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import policy_saver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

tempdir = os.getenv("TEST_TMPDIR", tempfile.gettempdir())

In [ ]:
#@test {"skip": true}
# Set up a virtual display for rendering OpenAI gym environments.
import xvfbwrapper
xvfbwrapper.Xvfb(1400, 900, 24).start()

## DQN 에이전트

이전 colab과 마찬가지로 DQN 에이전트를 설정하려고 합니다. 세부 사항은 colab의 핵심 부분이 아니므로 기본적으로 숨겨져 있지만, 'SHOW CODE'를 클릭하면 세부 사항을 볼 수 있습니다.

### 하이퍼 매개변수

In [ ]:
env_name = "CartPole-v1"

collect_steps_per_iteration = 100
replay_buffer_capacity = 100000

fc_layer_params = (100,)

batch_size = 64
learning_rate = 1e-3
log_interval = 5

num_eval_episodes = 10
eval_interval = 1000

### 환경

In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

### 에이전트

In [ ]:
#@title
q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

global_step = tf.compat.v1.train.get_or_create_global_step()

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=global_step)
agent.initialize()

### 데이터 수집

In [ ]:
#@title
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity)

collect_driver = dynamic_step_driver.DynamicStepDriver(
    train_env,
    agent.collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=collect_steps_per_iteration)

# Initial data collection
collect_driver.run()

# Dataset generates trajectories with shape [BxTx...] where
# T = n_step_update + 1.
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

iterator = iter(dataset)

### 에이전트 훈련하기

In [ ]:
#@title
# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

def train_one_iteration():

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_driver.run()

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience)

  iteration = agent.train_step_counter.numpy()
  print ('iteration: {0} loss: {1}'.format(iteration, train_loss.loss))

### 비디오 생성

In [ ]:
#@title
def embed_gif(gif_buffer):
  """Embeds a gif file in the notebook."""
  tag = '<img src="data:image/gif;base64,{0}"/>'.format(base64.b64encode(gif_buffer).decode())
  return IPython.display.HTML(tag)

def run_episodes_and_create_video(policy, eval_tf_env, eval_py_env):
  num_episodes = 3
  frames = []
  for _ in range(num_episodes):
    time_step = eval_tf_env.reset()
    frames.append(eval_py_env.render())
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = eval_tf_env.step(action_step.action)
      frames.append(eval_py_env.render())
  gif_file = io.BytesIO()
  imageio.mimsave(gif_file, frames, format='gif', fps=60)
  IPython.display.display(embed_gif(gif_file.getvalue()))

### 비디오 생성하기

비디오를 생성하여 정책의 성능을 확인합니다.

In [ ]:
print ('global_step:')
print (global_step)
run_episodes_and_create_video(agent.policy, eval_env, eval_py_env)

## Checkpointer 및 PolicySaver 설정

이제 Checkpointer와 PolicySaver를 사용할 준비가 되었습니다.

### Checkpointer


In [ ]:
checkpoint_dir = os.path.join(tempdir, 'checkpoint')
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=global_step
)

### PolicySaver

In [ ]:
policy_dir = os.path.join(tempdir, 'policy')
tf_policy_saver = policy_saver.PolicySaver(agent.policy)

### 한 번 반복 훈련하기

In [ ]:
#@test {"skip": true}
print('Training one iteration....')
train_one_iteration()

### 체크포인트에 저장하기

In [ ]:
train_checkpointer.save(global_step)

### 체크포인트 복원하기

체크포인트를 만들 때와 같은 방식으로 전체 객체 세트를 다시 만들어야 합니다.

In [ ]:
train_checkpointer.initialize_or_restore()
global_step = tf.compat.v1.train.get_global_step()

정책 저장 및 위치로 내보내기

In [ ]:
tf_policy_saver.save(policy_dir)

어떤 에이전트 또는 네트워크가 정책을 작성하는 데 사용되었는지 몰라도 정책을 로드할 수 있습니다. 따라서 정책 배포가 훨씬 쉬워집니다.

저장된 정책을 로드하고 수행 방법을 확인합니다.

In [ ]:
saved_policy = tf.saved_model.load(policy_dir)
run_episodes_and_create_video(saved_policy, eval_env, eval_py_env)

## 내보내기 및 가져오기

colab의 나머지는 checkpointer 및 정책 디렉토리를 내보내거나 가져오는 데 도움이 되므로 나중에 훈련을 계속하고 다시 훈련하지 않고도 모델을 배포할 수 있습니다.

이제 '한 번 반복 훈련하기'로 돌아가서 나중에 차이점을 이해할 수 있도록 몇 번 더 훈련할 수 있습니다. 약간 더 나은 결과가 나오기 시작하면 아래에서 계속합니다.

In [ ]:
#@title Create zip file and upload zip file (double-click to see the code)
def create_zip_file(dirname, base_filename):
  return shutil.make_archive(base_filename, 'zip', dirname)

def upload_and_unzip_file_to(dirname):
  if files is None:
    return
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    shutil.rmtree(dirname)
    zip_files = zipfile.ZipFile(io.BytesIO(uploaded[fn]), 'r')
    zip_files.extractall(dirname)
    zip_files.close()

체크포인트 디렉토리로부터 압축 파일을 생성합니다.

In [ ]:
train_checkpointer.save(global_step)
checkpoint_zip_filename = create_zip_file(checkpoint_dir, os.path.join(tempdir, 'exported_cp'))

zip 파일을 다운로드합니다.

In [ ]:
#@test {"skip": true}
if files is not None:
  files.download(checkpoint_zip_filename) # try again if this fails: https://github.com/googlecolab/colabtools/issues/469

일정 시간(10~15회) 훈련한 후 체크포인트 zip 파일을 다운로드하고 "Runtime &gt; Restart and run all"로 이동하여 훈련을 재설정하고 이 셀로 돌아옵니다. 이제 다운로드한 zip 파일을 업로드하고 훈련을 계속할 수 있습니다.

In [ ]:
#@test {"skip": true}
upload_and_unzip_file_to(checkpoint_dir)
train_checkpointer.initialize_or_restore()
global_step = tf.compat.v1.train.get_global_step()

체크포인트 디렉토리를 업로드한 후에는 '한 번 반복 훈련하기'로 돌아가서 훈련을 계속하거나 '비디오 생성하기'로 돌아가서 로드된 정책의 성능을 확인합니다.

또는 정책(모델)을 저장하고 복원할 수 있습니다. 체크포인트와 달리 훈련을 계속할 수는 없지만, 모델을 계속 배포할 수 있습니다. 다운로드한 파일은 checkpointer의 파일보다 훨씬 작습니다.

In [ ]:
tf_policy_saver.save(policy_dir)
policy_zip_filename = create_zip_file(policy_dir, os.path.join(tempdir, 'exported_policy'))

In [ ]:
#@test {"skip": true}
if files is not None:
  files.download(policy_zip_filename) # try again if this fails: https://github.com/googlecolab/colabtools/issues/469

다운로드한 정책 디렉토리(exported_policy.zip)를 업로드하고 저장된 정책의 성능을 확인합니다.

In [ ]:
#@test {"skip": true}
upload_and_unzip_file_to(policy_dir)
saved_policy = tf.saved_model.load(policy_dir)
run_episodes_and_create_video(saved_policy, eval_env, eval_py_env)


## SavedModelPyTFEagerPolicy

TF 정책을 사용하지 않으려면, `py_tf_eager_policy.SavedModelPyTFEagerPolicy`를 사용하여 Phython env로 saved_model을 직접 사용할 수도 있습니다.

즉시 모드가 활성화된 경우에만 동작합니다.

In [ ]:
eager_py_policy = py_tf_eager_policy.SavedModelPyTFEagerPolicy(
    policy_dir, eval_py_env.time_step_spec(), eval_py_env.action_spec())

# Note that we're passing eval_py_env not eval_env.
run_episodes_and_create_video(eager_py_policy, eval_py_env, eval_py_env)

## 정책을 TFLite로 변환하기

자세한 내용은 [TensorFlow Lite 변환기](https://www.tensorflow.org/lite/convert)를 참조하십시오.

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(policy_dir, signature_keys=["action"])
tflite_policy = converter.convert()
with open(os.path.join(tempdir, 'policy.tflite'), 'wb') as f:
  f.write(tflite_policy)

### TFLite 모델에서 추론 실행하기

자세한 내용은 [TensorFlow Lite 추론](https://tensorflow.org/lite/guide/inference)을 참조하십시오.

In [ ]:
import numpy as np
interpreter = tf.lite.Interpreter(os.path.join(tempdir, 'policy.tflite'))

policy_runner = interpreter.get_signature_runner()
print(policy_runner._inputs)

In [ ]:
policy_runner(**{
    '0/discount':tf.constant(0.0),
    '0/observation':tf.zeros([1,4]),
    '0/reward':tf.constant(0.0),
    '0/step_type':tf.constant(0)})